In [ ]:
import tensorflow as tf

# Get the GPU device name.
device_name = tf.test.gpu_device_name()

# The device name should look like the following:
if device_name == '/device:GPU:0':
    print('Found GPU at: {}'.format(device_name))
else:
    raise SystemError('GPU device not found')


Found GPU at: /device:GPU:0


In [ ]:
import torch

# If there's a GPU available...
if torch.cuda.is_available():

    # Tell PyTorch to use the GPU.
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")


There are 1 GPU(s) available.
We will use the GPU: Tesla T4


In [ ]:

!pip install transformers


In [ ]:
import pandas as pd



In [ ]:
# Load the dataset into a pandas dataframe.
df = pd.read_csv("/content/Amazon Reviews Validation Data.csv",encoding='latin-1',header=None,names=['sentence', 'label'])



In [ ]:
len(df)

1182

In [ ]:
df['label']=df['label'].replace('Aspect', 1)


In [ ]:
df['label']=df['label'].replace('Safety', 1)
df['label']=df['label'].replace('Adaptability', 2)
df['label']=df['label'].replace('Performance', 2)
df['label']=df['label'].replace('Use Efficiency', 2)
df['label']=df['label'].replace('Aesthetics', 3)
df['label']=df['label'].replace('Ease of Use', 4)
df['label']=df['label'].replace('Ease of Reprocessing', 4)
df['label']=df['label'].replace('Ease of Storage', 4)
df['label']=df['label'].replace('Ergonomics',4)
df['label']=df['label'].replace('Price', 5)
df['label']=df['label'].replace('Durability',6)
df['label']=df['label'].replace('Interference', 6)

<ipython-input-8-bef1ffa1796e>:12: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['label']=df['label'].replace('Interference', 6)


In [ ]:
df.drop(df.index[0], inplace=True)

In [ ]:
df.head()


,sentence,label
1,The settings are perfect for all hair types,2
2,with lots of settings,2
3,heating element has died,6
4,"Also, theÊdiffuserÊdoes not stay on at all.",4
5,and it won't stay on it at all. Just flys off....,4


In [ ]:
len(df)

1181

In [ ]:

import pandas as pd

# Load the dataset into a pandas dataframe.
df_beer = pd.read_csv("/content/test.txt", delimiter='\t', header=None, names=['sentence'])
df_label = pd.read_csv("/content/test_label.txt", delimiter='\t', header=None, names=['label'])

# Report the number of sentences.
print('Number of test sentences: {:,}\n'.format(df_beer.shape[0]))




Number of test sentences: 9,236



In [ ]:
df_label

,label
0,NaN
1,look
2,smell
3,taste
4,taste
...,...
9231,taste
9232,feel
9233,overall
9234,overall


In [ ]:
df_label.label.unique()

array([nan, 'look', 'smell', 'taste', 'feel', 'overall'], dtype=object)

In [ ]:
df_label.fillna(6, inplace=True)


In [ ]:
df_label['label']=df_label['label'].replace('look', 1)

In [ ]:
df_label['label']=df_label['label'].replace('smell', 2)

In [ ]:
df_label['label']=df_label['label'].replace('taste', 3)
df_label['label']=df_label['label'].replace('feel', 4)
df_label['label']=df_label['label'].replace('overall', 5)

<ipython-input-18-11c3038e02f7>:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_label['label']=df_label['label'].replace('overall', 5)


In [ ]:
df_beer=pd.concat([df_beer,df_label], axis=1)

In [ ]:
# add 2 pandas data frames
df = pd.concat([df[:800],df_beer[:7000]], ignore_index=True)

In [ ]:
df=df.dropna()

In [ ]:
sentences = df.sentence.values
labels = df.label.values

In [ ]:
from transformers import BertTokenizer

# Load the BERT tokenizer.
print('Loading BERT tokenizer...')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

Loading BERT tokenizer...


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
# Print the original sentence.
print(' Original: ', sentences[0])

# Print the sentence split into tokens.
print('Tokenized: ', tokenizer.tokenize(sentences[0]))

# Print the sentence mapped to token ids.
print('Token IDs: ', tokenizer.convert_tokens_to_ids(tokenizer.tokenize(sentences[0])))


 Original:  The settings are perfect for all hair types
Tokenized:  ['the', 'settings', 'are', 'perfect', 'for', 'all', 'hair', 'types']
Token IDs:  [1996, 10906, 2024, 3819, 2005, 2035, 2606, 4127]


In [ ]:
max_len = 0

# For every sentence...
for sent in sentences:

    # Tokenize the text and add `[CLS]` and `[SEP]` tokens.
    input_ids = tokenizer.encode(sent, add_special_tokens=True)

    # Update the maximum sentence length.
    max_len = max(max_len, len(input_ids))

print('Max sentence length: ', max_len)


Max sentence length:  69


In [ ]:
# Tokenize all of the sentences and map the tokens to thier word IDs.
input_ids = []
attention_masks = []

# For every sentence...
for sent in sentences:
    # `encode_plus` will:
    #   (1) Tokenize the sentence.
    #   (2) Prepend the `[CLS]` token to the start.
    #   (3) Append the `[SEP]` token to the end.
    #   (4) Map tokens to their IDs.
    #   (5) Pad or truncate the sentence to `max_length`
    #   (6) Create attention masks for [PAD] tokens.
    encoded_dict = tokenizer.encode_plus(
                        sent,                      # Sentence to encode.
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                        max_length = 256,           # Pad & truncate all sentences.
                        pad_to_max_length = True,
                        return_attention_mask = True,   # Construct attn. masks.
                        return_tensors = 'pt',     # Return pytorch tensors.
                   )

    # Add the encoded sentence to the list.
    input_ids.append(encoded_dict['input_ids'])

    # And its attention mask (simply differentiates padding from non-padding).
    attention_masks.append(encoded_dict['attention_mask'])

# Convert the lists into tensors.



Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2870: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [ ]:
# If input_ids and attention_masks are already lists of tensors, use clone().detach() to avoid the warning.
input_ids = [ids.clone().detach() if isinstance(ids, torch.Tensor) else torch.tensor(ids) for ids in input_ids]
attention_masks = [mask.clone().detach() if isinstance(mask, torch.Tensor) else torch.tensor(mask) for mask in attention_masks]

# Check dimensions and reshape if necessary to avoid zero-dimensional tensors
input_ids = [ids.unsqueeze(0) if ids.dim() == 0 else ids for ids in input_ids]
attention_masks = [mask.unsqueeze(0) if mask.dim() == 0 else mask for mask in attention_masks]

# Concatenate the list of tensors along the desired dimension
input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)


# Print the first sentence, now as a list of IDs
print(input_ids[0])

tensor([  101,  1996, 10906,  2024,  3819,  2005,  2035,  2606,  4127,   102,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0, 

In [ ]:

import numpy as np

# Assuming `labels` is a NumPy array; check its type and convert if necessary
if isinstance(labels, np.ndarray):
    # Convert to a numeric type if it contains numeric data; handle non-numeric data accordingly
    try:
        labels = labels.astype(np.int64)  # Convert to integers
    except ValueError:
        # Handle non-numeric cases (e.g., string labels or mixed types)
        # For example, convert string labels to integers or map them to numerical indices
        unique_labels = list(set(labels))  # Get unique labels
        label_to_int = {label: i for i, label in enumerate(unique_labels)}  # Map labels to integers
        labels = np.array([label_to_int[label] for label in labels])  # Convert labels to integers

# Now convert to a PyTorch tensor
labels = torch.tensor(labels, dtype=torch.long)

# Print the first label to check the conversion
print(labels[0])

tensor(2)


In [ ]:
# Print sentence 0, now as a list of IDs.
print('Original: ', sentences[0])
print('Token IDs:', input_ids[0])

Original:  The settings are perfect for all hair types
Token IDs: tensor([  101,  1996, 10906,  2024,  3819,  2005,  2035,  2606,  4127,   102,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0

In [ ]:
from torch.utils.data import TensorDataset, random_split

# Debugging: Check the sizes of the tensors
print("Size of input_ids:", input_ids.size())
print("Size of attention_masks:", attention_masks.size())
print("Size of labels:", labels.size())

# Ensure all tensors have the same length
min_size = min(input_ids.size(0), attention_masks.size(0), labels.size(0))

# Truncate tensors to the minimum size if necessary
input_ids = input_ids[:min_size]
attention_masks = attention_masks[:min_size]
labels = labels[:min_size]

# Combine the training inputs into a TensorDataset.
dataset = TensorDataset(input_ids, attention_masks, labels)

# Create a 90-10 train-validation split.
train_size = int(0.7 * len(dataset))
val_size = len(dataset) - train_size

# Divide the dataset by randomly selecting samples.
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

print('{:>5,} training samples'.format(train_size))
print('{:>5,} validation samples'.format(val_size))

Size of input_ids: torch.Size([7800, 256])
Size of attention_masks: torch.Size([7800, 256])
Size of labels: torch.Size([7800])
5,460 training samples
2,340 validation samples


In [ ]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

# The DataLoader needs to know our batch size for training, so we specify it
# here. For fine-tuning BERT on a specific task, the authors recommend a batch
# size of 16 or 32.
batch_size = 32

# Create the DataLoaders for our training and validation sets.
# We'll take training samples in random order.
train_dataloader = DataLoader(
            train_dataset,  # The training samples.
            sampler = RandomSampler(train_dataset), # Select batches randomly
            batch_size = batch_size # Trains with this batch size.
        )

# For validation the order doesn't matter, so we'll just read them sequentially.
validation_dataloader = DataLoader(
            val_dataset, # The validation samples.
            sampler = SequentialSampler(val_dataset), # Pull out batches sequentially.
            batch_size = batch_size # Evaluate with this batch size.
        )


In [ ]:
from transformers import BertForSequenceClassification, AdamW, BertConfig

# Load BertForSequenceClassification, the pretrained BERT model with a single
# linear classification layer on top.
model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased", # Use the 12-layer BERT model, with an uncased vocab.
    num_labels = 7, # The number of output labels--2 for binary classification.
                    # You can increase this for multi-class tasks.
    output_attentions = False, # Whether the model returns attentions weights.
    output_hidden_states = False, # Whether the model returns all hidden-states.
)

# Tell pytorch to run this model on the GPU.
model.cuda()


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [ ]:
# Get all of the model's parameters as a list of tuples.
params = list(model.named_parameters())

print('The BERT model has {:} different named parameters.\n'.format(len(params)))

print('==== Embedding Layer ====\n')

for p in params[0:5]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

print('\n==== First Transformer ====\n')

for p in params[5:21]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

print('\n==== Output Layer ====\n')

for p in params[-4:]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))


The BERT model has 201 different named parameters.

==== Embedding Layer ====

bert.embeddings.word_embeddings.weight                  (30522, 768)
bert.embeddings.position_embeddings.weight                (512, 768)
bert.embeddings.token_type_embeddings.weight                (2, 768)
bert.embeddings.LayerNorm.weight                              (768,)
bert.embeddings.LayerNorm.bias                                (768,)

==== First Transformer ====

bert.encoder.layer.0.attention.self.query.weight          (768, 768)
bert.encoder.layer.0.attention.self.query.bias                (768,)
bert.encoder.layer.0.attention.self.key.weight            (768, 768)
bert.encoder.layer.0.attention.self.key.bias                  (768,)
bert.encoder.layer.0.attention.self.value.weight          (768, 768)
bert.encoder.layer.0.attention.self.value.bias                (768,)
bert.encoder.layer.0.attention.output.dense.weight        (768, 768)
bert.encoder.layer.0.attention.output.dense.bias              (

In [ ]:
from transformers import AdamW, BertForSequenceClassification, BertConfig #Import AdamW from transformers
# Note: AdamW is a class from the huggingface library (as opposed to pytorch)
# I believe the 'W' stands for 'Weight Decay fix"
optimizer = AdamW(model.parameters(),
                  lr = 2e-5, # args.learning_rate - default is 5e-5, our notebook had 2e-5
                  eps = 1e-8 # args.adam_epsilon  - default is 1e-8.
                )

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
from transformers import get_linear_schedule_with_warmup

# Number of training epochs. The BERT authors recommend between 2 and 4.
# We chose to run for 4, but we'll see later that this may be over-fitting the
# training data.
epochs = 4

# Total number of training steps is [number of batches] x [number of epochs].
# (Note that this is not the same as the number of training samples).
total_steps = len(train_dataloader) * epochs

# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(optimizer,
                                            num_warmup_steps = 0, # Default value in run_glue.py
                                            num_training_steps = total_steps)


In [ ]:
import numpy as np

# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)


In [ ]:
import time
import datetime

def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))

    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))


In [ ]:
import random
import numpy as np
import torch

# This training code is based on the `run_glue.py` script here:
# https://github.com/huggingface/transformers/blob/5bfcd0485ece086ebcbed2d008813037968a9e58/examples/run_glue.py#L128

# Set the seed value all over the place to make this reproducible.
seed_val = 44

random.seed(seed_val)
np.random.seed(seed_val)

# Check if CUDA is available before setting the seed
if torch.cuda.is_available():
  torch.manual_seed(seed_val)
  torch.cuda.manual_seed_all(seed_val)

# We'll store a number of quantities such as training and validation loss,
# validation accuracy, and timings.
training_stats_2 = []

# Measure the total training time for the whole run.
total_t0 = time.time()

# For each epoch...
for epoch_i in range(0, epochs):

    # ========================================
    #               Training
    # ========================================

    # Perform one full pass over the training set.

    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # Measure how long the training epoch takes.
    t0 = time.time()

    # Reset the total loss for this epoch.
    total_train_loss_2 = 0

    # Put the model into training mode. Don't be mislead--the call to
    # `train` just changes the *mode*, it doesn't *perform* the training.
    # `dropout` and `batchnorm` layers behave differently during training
    # vs. test (source: https://stackoverflow.com/questions/51433378/what-does-model-train-do-in-pytorch)
    model.train()

    # For each batch of training data...
    for step, batch in enumerate(train_dataloader):

        # Progress update every 40 batches.
        if step % 40 == 0 and not step == 0:
            # Calculate elapsed time in minutes.
            elapsed = format_time(time.time() - t0)

            # Report progress.
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        # Unpack this training batch from our dataloader.
        #
        # As we unpack the batch, we'll also copy each tensor to the GPU using the
        # `to` method.
        #
        # `batch` contains three pytorch tensors:
        #   [0]: input ids
        #   [1]: attention masks
        #   [2]: labels
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        # Always clear any previously calculated gradients before performing a
        # backward pass. PyTorch doesn't do this automatically because
        # accumulating the gradients is "convenient while training RNNs".
        # (source: https://stackoverflow.com/questions/48001598/why-do-we-need-to-call-zero-grad-in-pytorch)
        model.zero_grad()

        # Perform a forward pass (evaluate the model on this training batch).
        # The documentation for this `model` function is here:
        # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
        # It returns different numbers of parameters depending on what arguments
        # arge given and what flags are set. For our useage here, it returns
        # the loss (because we provided labels) and the "logits"--the model
        # outputs prior to activation.
        # Debugging: Print the shape and type of labels
        # Debugging: Print the number of classes the model expects
        print("Model expects {} classes.".format(model.config.num_labels))

        # Debugging: Print the minimum and maximum label values in the batch
        print("Labels - Min:", b_labels.min().item(), "Max:", b_labels.max().item())

        # Get the number of classes from the model configuration
        num_labels = model.config.num_labels

        # Check and fix labels if necessary
        if b_labels.min() < 0 or b_labels.max() >= num_labels:
            print("Correcting label range...")

            # Option 1: Clip the labels to be within the range (if appropriate for your task)
            b_labels = torch.clamp(b_labels, min=0, max=num_labels - 1)

            # Option 2: Map the labels to valid range if they are completely out of bounds (example mapping)
            # label_mapping = {your_mapping_logic_here}
            # b_labels = torch.tensor([label_mapping[label.item()] for label in b_labels])

        # Ensure the corrected labels are within the range
        assert b_labels.min() >= 0 and b_labels.max() < num_labels, "Labels are still out of bounds after correction."

        # Continue with forward pass
        output = model(
            b_input_ids,
            token_type_ids=None,
            attention_mask=b_input_mask,
            labels=b_labels
        )

        # The model outputs a tuple (loss, logits)
        loss = output[0]
        logits = output[1]

        # Debugging: Check the shape and type of logits
        print("Logits shape:", logits.shape)
        print("Logits type:", logits.dtype)

        # Accumulate the training loss over all of the batches so that we can
        # calculate the average loss at the end. `loss` is a Tensor containing a
        # single value; the `.item()` function just returns the Python value from the tensor.
        total_train_loss_2 += loss.item()

        # Perform a backward pass to calculate the gradients.
        loss.backward()

        # Clip the norm of the gradients to 1.0.
        # This is to help prevent the "exploding gradients" problem.
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # Update parameters and take a step using the computed gradient.
        # The optimizer dictates the "update rule"--how the parameters are
        # modified based on their gradients, the learning rate, etc.
        optimizer.step()

        # Update the learning rate.
        scheduler.step()

    # Calculate the average loss over all of the batches.
    avg_train_loss = total_train_loss_2 / len(train_dataloader)

    # Measure how long this epoch took.
    training_time = format_time(time.time() - t0)

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(training_time))

    # ========================================
    #               Validation
    # ========================================
    # After the completion of each training epoch, measure our performance on
    # our validation set.

    print("")
    print("Running Validation...")

    t0 = time.time()

    # Put the model in evaluation mode--the dropout layers behave differently
    # during evaluation.
    model.eval()

    # Tracking variables
    total_eval_accuracy_2 = 0
    total_eval_loss_2 = 0
    nb_eval_steps = 0

    # Evaluate data for one epoch
    for batch in validation_dataloader:

        # Unpack this training batch from our dataloader.
        #
        # As we unpack the batch, we'll also copy each tensor to the GPU using
        # the `to` method.
        #
        # `batch` contains three pytorch tensors:
        #   [0]: input ids
        #   [1]: attention masks
        #   [2]: labels
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        # Tell pytorch not to bother with constructing the compute graph during
        # the forward pass, since this is only needed for backprop (training).
        print("Model expects {} classes.".format(model.config.num_labels))

        # Debugging: Print the minimum and maximum label values in the batch
        print("Labels - Min:", b_labels.min().item(), "Max:", b_labels.max().item())

        # Get the number of classes from the model configuration
        num_labels = model.config.num_labels

        # Check and fix labels if necessary
        if b_labels.min() < 0 or b_labels.max() >= num_labels:
            print("Correcting label range...")

            # Option 1: Clip the labels to be within the range (if appropriate for your task)
            b_labels = torch.clamp(b_labels, min=0, max=num_labels - 1)

            # Option 2: Map the labels to valid range if they are completely out of bounds (example mapping)
            # label_mapping = {your_mapping_logic_here}
            # b_labels = torch.tensor([label_mapping[label.item()] for label in b_labels])

        # Ensure the corrected labels are within the range
        assert b_labels.min() >= 0 and b_labels.max() < num_labels, "Labels are still out of bounds after correction."

        with torch.no_grad():

            # Forward pass, calculate logit predictions.
            # token_type_ids is the same as the "segment ids", which
            # differentiates sentence 1 and 2 in 2-sentence tasks.
            # The documentation for this `model` function is here:
            # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
            # Get the "logits" output by the model. The "logits" are the output
            # values prior to applying an activation function like the softmax.
            # Continue with forward pass
            output = model(
                b_input_ids,
                token_type_ids=None,
                attention_mask=b_input_mask,
                labels=b_labels
            )

            # The model outputs a tuple (loss, logits)
            loss = output[0]
            logits = output[1]
            # Debugging: Check the shape and type of logits
            print("Logits shape:", logits.shape)
            print("Logits type:", logits.dtype)

            # Accumulate the training loss over all of the batches so that we can
            # calculate the average loss at the end. `loss` is a Tensor containing a
            # single value; the `.item()` function just returns the Python value from the tensor.
            total_eval_loss_2 += loss.item()


        # Move logits and labels to CPU
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()

        # Calculate the accuracy for this batch of test sentences, and
        # accumulate it over all batches.
        total_eval_accuracy_2 += flat_accuracy(logits, label_ids)


    # Report the final accuracy for this validation run.
    avg_val_accuracy = total_eval_accuracy_2 / len(validation_dataloader)
    print("  Accuracy: {0:.2f}".format(avg_val_accuracy))

    # Calculate the average loss over all of the batches.
    avg_val_loss = total_eval_loss_2 / len(validation_dataloader)

    # Measure how long the validation run took.
    validation_time = format_time(time.time() - t0)

    print("  Validation Loss: {0:.2f}".format(avg_val_loss))
    print("  Validation took: {:}".format(validation_time))

    # Record all statistics from this epoch.
    training_stats_2.append(
        {
            'epoch': epoch_i + 1,
            'Training Loss': avg_train_loss,
            'Valid. Loss': avg_val_loss,
            'Valid. Accur.': avg_val_accuracy,
            'Training Time': training_time,
            'Validation Time': validation_time
        }
    )

print("")
print("Training complete!")

print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))



======== Epoch 1 / 4 ========
Training...
Model expects 7 classes.
Labels - Min: 1 Max: 6
Logits shape: torch.Size([32, 7])
Logits type: torch.float32
Model expects 7 classes.
Labels - Min: 1 Max: 6
Logits shape: torch.Size([32, 7])
Logits type: torch.float32
Model expects 7 classes.
Labels - Min: 1 Max: 6
Logits shape: torch.Size([32, 7])
Logits type: torch.float32
Model expects 7 classes.
Labels - Min: 1 Max: 6
Logits shape: torch.Size([32, 7])
Logits type: torch.float32
Model expects 7 classes.
Labels - Min: 1 Max: 6
Logits shape: torch.Size([32, 7])
Logits type: torch.float32
Model expects 7 classes.
Labels - Min: 1 Max: 6
Logits shape: torch.Size([32, 7])
Logits type: torch.float32
Model expects 7 classes.
Labels - Min: 1 Max: 6
Logits shape: torch.Size([32, 7])
Logits type: torch.float32
Model expects 7 classes.
Labels - Min: 1 Max: 6
Logits shape: torch.Size([32, 7])
Logits type: torch.float32
Model expects 7 classes.
Labels - Min: 1 Max: 6
Logits shape: torch.Size([32, 7])
Log

In [ ]:
training_stats_2

[{'epoch': 1,
  'Training Loss': 1.1991161547209088,
  'Valid. Loss': 0.7682113764253823,
  'Valid. Accur.': 0.7487331081081081,
  'Training Time': '0:03:35',
  'Validation Time': '0:00:32'},
 {'epoch': 2,
  'Training Loss': 0.6368883523153283,
  'Valid. Loss': 0.6697023454550151,
  'Valid. Accur.': 0.7744932432432432,
  'Training Time': '0:03:34',
  'Validation Time': '0:00:32'},
 {'epoch': 3,
  'Training Loss': 0.4728632009865945,
  'Valid. Loss': 0.6638756374249587,
  'Valid. Accur.': 0.7791385135135135,
  'Training Time': '0:03:34',
  'Validation Time': '0:00:32'},
 {'epoch': 4,
  'Training Loss': 0.3815568919941696,
  'Valid. Loss': 0.6741903251087343,
  'Valid. Accur.': 0.78125,
  'Training Time': '0:03:34',
  'Validation Time': '0:00:32'}]

# tried to use amazon dataset to predict the output vales

In [ ]:
df_2=df_beer[7001:]

In [ ]:
df_2 = df_2.dropna()

In [ ]:
# Create sentence and label lists
sentences = df_2.sentence.values
labels = df_2.label.values

In [ ]:
import torch
from torch.utils.data import DataLoader, TensorDataset, SequentialSampler
import numpy as np

input_ids = []
attention_masks = []

# Tokenize and encode sentences
for sent in sentences:
    encoded_dict = tokenizer.encode_plus(
                        sent,
                        add_special_tokens = True,
                        max_length = 64,
                        padding = 'max_length',
                        truncation = True,
                        return_attention_mask = True,
                        return_tensors = 'pt',
                   )
    input_ids.append(encoded_dict['input_ids'])
    attention_masks.append(encoded_dict['attention_mask'])

# Convert the lists into tensors.
input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)



In [ ]:

labels_array = np.array(labels)




In [ ]:
# Now, convert the numpy array to a PyTorch tensor
labels_tensor = torch.tensor(labels_array, dtype=torch.int64)

# Print to check the tensor
print(labels_tensor)

tensor([5, 1, 1,  ..., 5, 5, 5])


In [ ]:
import torch
from torch.utils.data import TensorDataset, DataLoader, SequentialSampler

# Conversion of labels from numpy array to PyTorch tensor
labels = torch.tensor(labels)  # Ensure this is not causing a type error

# Check for size compatibility again with the corrected tensor
print("input_ids size:", input_ids.size(0))
print("attention_masks size:", attention_masks.size(0))
print("labels size:", labels.size(0))

# Proceed if sizes are compatible
if input_ids.size(0) == attention_masks.size(0) == labels.size(0):
    print("All tensors are compatible.")
    prediction_data = TensorDataset(input_ids, attention_masks, labels)
    prediction_sampler = SequentialSampler(prediction_data)
    prediction_dataloader = DataLoader(prediction_data, sampler=prediction_sampler, batch_size=batch_size)
else:
    print("There is a size mismatch among tensors.")

input_ids size: 2235
attention_masks size: 2235
labels size: 2235
All tensors are compatible.


In [ ]:
import os
import torch
import numpy as np

# Set CUDA_LAUNCH_BLOCKING for better error messages
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

# Check if CUDA is available and set the device accordingly
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# Move the model to the chosen device
model = model.to(device)
model.eval()  # Set the model to evaluation mode

total_eval_loss = 0
total_eval_accuracy = 0

loss_fn = torch.nn.CrossEntropyLoss()

def flat_accuracy(preds, labels):
    # Using PyTorch functions for efficiency
    pred_flat = torch.argmax(preds, dim=1)
    correct = (pred_flat == labels).sum().item()
    total = labels.size(0)
    return correct / total

for batch in prediction_dataloader:
    # Move each batch of data to the same device as the model
    b_input_ids = batch[0].to(device)
    b_attention_mask = batch[1].to(device)
    b_labels = batch[2].to(device).long()  # Ensure labels are of type LongTensor

    # Print label values for debugging
    print("Labels:", b_labels)
    print("Labels min value:", b_labels.min().item())
    print("Labels max value:", b_labels.max().item())
    print("Unique labels:", b_labels.unique())

    # Adjust labels to start from 0 if necessary
    if b_labels.min() >= 1:
        print("Adjusting labels to start from 0")
        b_labels = b_labels - 1

    # Set num_classes based on adjusted labels
    num_classes = b_labels.max().item() + 1
    model.config.num_labels = num_classes

    # Ensure labels are within the expected range
    assert b_labels.min() >= 0 and b_labels.max() < num_classes, "Labels out of range!"

    with torch.no_grad():  # Temporarily disable gradient calculation
        # Forward pass: compute the output of the model
        outputs = model(b_input_ids, attention_mask=b_attention_mask)
        logits = outputs.logits if hasattr(outputs, 'logits') else outputs

        # Compute loss
        loss = loss_fn(logits, b_labels)

    # Move outputs to CPU before printing if needed
    print("Logits:", logits.detach().cpu())
    print("Loss:", loss.item())
    print("Labels:", b_labels.detach().cpu())

    # Calculate the accuracy for this batch of test sentences
    batch_accuracy = flat_accuracy(logits, b_labels)
    total_eval_accuracy += batch_accuracy
    total_eval_loss += loss.item()



Using device: cuda
Labels: tensor([5, 1, 1, 1, 3, 3, 3, 3, 3, 4, 4, 3, 5, 5, 5, 6, 6, 6, 6, 6, 6, 5, 5, 5,
        2, 6, 3, 5, 1, 1, 2, 2], device='cuda:0')
Labels min value: 1
Labels max value: 6
Unique labels: tensor([1, 2, 3, 4, 5, 6], device='cuda:0')
Adjusting labels to start from 0
Logits: tensor([[-1.5446, -2.2678, -0.9699,  1.4062, -0.6277,  4.2893,  0.0254],
        [-0.9113,  5.0404, -0.4638, -1.4179, -0.7687, -1.9553, -0.4053],
        [-0.8714,  5.0790, -0.6071, -1.2357, -1.0595, -1.7084, -0.4196],
        [-0.8846,  5.0038, -0.8771, -1.1874, -1.1985, -1.4661, -0.3248],
        [-1.6798, -0.4952,  2.7509,  2.4267, -1.6685, -1.2165, -0.6532],
        [-1.5804, -1.4002,  3.3072,  2.5168, -1.1159, -1.2744, -0.9208],
        [-1.5960, -1.3311,  3.0254,  2.9559, -1.6319, -1.0621, -0.9228],
        [-1.5911, -1.2052,  3.5571,  2.2384, -1.4610, -1.3650, -0.7087],
        [-1.9802, -1.5205,  2.2831,  1.1013,  0.1089, -0.2347,  0.0206],
        [-0.1804, -0.8411, -0.3525, -0.3167,  

In [ ]:
# Report the final accuracy for this validation run
avg_val_accuracy = total_eval_accuracy / len(prediction_dataloader)
print("Accuracy: {0:.2f}".format(avg_val_accuracy))
print("Average testing loss: {0:.2f}".format(total_eval_loss / len(prediction_dataloader)))

Accuracy: 0.05
Average testing loss: 4.44


In [ ]:
# Assuming 'model' is your fine-tuned BERT model
output_dir = "./my_finetuned_bert_model"
model.save_pretrained(output_dir)

# If you're using a specific tokenizer, save that too
tokenizer.save_pretrained(output_dir)

('./my_finetuned_bert_model/tokenizer_config.json',
 './my_finetuned_bert_model/special_tokens_map.json',
 './my_finetuned_bert_model/vocab.txt',
 './my_finetuned_bert_model/added_tokens.json')